In [ ]:
%load_ext autoreload
%autoreload 2

# Processing Sensitivity Analysis Outputs

In [ ]:
import os
from pathlib import Path


ROOT = Path(os.getcwd()).parent
while not ROOT.joinpath(".git").exists():
    ROOT = ROOT.parent

# add the root to the python path
import sys
import dotenv
from omegaconf import OmegaConf

sys.path.append(str(ROOT))

# load the environment variables
dotenv.load_dotenv(ROOT.joinpath(".env"))

In [ ]:
import polars as pl

from sumo_pipelines.utils.config_helpers import (
    create_custom_resolvers,
)
from sumo_pipelines.utils.file_helpers import walk_directory

create_custom_resolvers()

## Load the Data

In [ ]:
experiment_root = (
    ROOT / "tmp/SobolSensitivityAnalysisCarsTrucks-1024/11.28.2023_15.57.14"
)

### Build A Dataframe from Results

In [ ]:
results_df_path = experiment_root / "results.parquet"

if not results_df_path.exists():
    confs = list(
        walk_directory(
            experiment_root,
        )
    )
    confs.sort(key=lambda x: int(x.Metadata.run_id))

    results_df = (
        pl.from_records(
            [
                {
                    "run_id": conf.Metadata.run_id,
                    "calibration_passed": (
                        conf.Pipeline.pipeline[0].consumers[7].config.calibration_passed
                    ),
                    **OmegaConf.to_container(
                        conf.Pipeline.pipeline[0].consumers[8].config
                    ),
                    **{
                        "total_vehicles_emissions": conf.Blocks.FuelTotalConfig.total_vehicles,
                        "total_energy": conf.Blocks.FuelTotalConfig.total_energy,
                    },
                    # **OmegaConf.to_container(conf.Blocks.FuelTotalConfig),
                }
                for conf in confs
            ]
        )
        .with_columns(
            pl.col("run_id").cast(pl.UInt32),
        )
        .drop(["detector_file", "trip_info_file", "warmup_time"])
    )

    # save the results
    results_df.write_parquet(experiment_root / "results.parquet")

else:
    results_df = pl.read_parquet(results_df_path)

repr_conf = OmegaConf.load(experiment_root / "0" / "config.yaml")

In [ ]:
print(experiment_root / "results.parquet")

### Build a DataFrame from Sobol Sequence

In [ ]:
sample_df = pl.read_parquet(
    experiment_root / repr_conf.Blocks.SobolSequenceConfig.save_path
).with_row_count("run_id")

In [ ]:
# results_df["total_energy"].head()

### Update the Fuel Amount

In [ ]:
from src.sa_helpers.metrics import (
    SUMO_GASOLINE_GRAM_PER_LITER,
    SUMO_GASOLINE_GRAM_TO_JOULE,
)

results_df = results_df.with_columns(
    total_fuel_l_cropped=pl.col("total_energy")
    / SUMO_GASOLINE_GRAM_TO_JOULE
    / SUMO_GASOLINE_GRAM_PER_LITER
).with_columns(
    cropped_per_vehicle_fuel=pl.col("total_fuel_l_cropped")
    / pl.col("total_vehicles_emissions")
)

In [ ]:
results_df["run_id"].max() + 1

### Join the DataFrame

In [ ]:
results_df = results_df.join(sample_df, on="run_id")
results_df.head()

In [ ]:
1 - results_df["calibration_passed"].mean()

In [ ]:
results_df.filter(
    ~pl.col("calibration_passed"),
).sort(
    "average_fc",
    descending=True,
)

In [ ]:
results_df["average_travel_time"]

In [ ]:
# pl.read_parquet(experiment_root / "11615" / "usdot_res.parquet")

### Read in all the calibration results

In [ ]:
df = pl.concat(
    pl.scan_parquet(
        experiment_root / f"{i}" / "usdot_res.parquet",
    ).with_columns(
        run_id=pl.lit(i, dtype=pl.UInt32),
    )
    for i in results_df["run_id"]
).collect()

In [ ]:
df.filter(pl.col("BDSE") > pl.col("BDSE_Threshold"))

In [ ]:
results_df = results_df.join(
    df.group_by("run_id").agg(pl.col("BDAE").mean()), on="run_id"
)

## Perform Sensitivity Analysis

In [ ]:
from SALib.analyze import sobol
from SALib.sample import sobol as sobol_sample
from sumo_pipelines.blocks.producers.config import SobolSequenceConfig

In [ ]:
problem = SobolSequenceConfig.build_sobol_dict(repr_conf.Blocks.SobolSequenceConfig)
second_order = repr_conf.Blocks.SobolSequenceConfig.calc_second_order


def get_sa_results(results_df, problem, column, second_order=False):
    return sobol.analyze(
        problem, results_df[column].to_numpy(), calc_second_order=second_order
    )

### Verify that Everything is in Right Order

In [ ]:
import numpy as np

sobol_sampled = sobol_sample.sample(
    problem,
    N=repr_conf.Blocks.SobolSequenceConfig.N,
    calc_second_order=second_order,
    seed=repr_conf.Metadata.random_seed,
)

assert np.allclose(sobol_sampled, sample_df[problem["names"]].to_numpy())

### Perform Sensitivity Analysis

In [ ]:
target_cols = set(results_df.columns).intersection(
    set(repr_conf.Pipeline.pipeline[0].consumers[8].config.keys())
) | set(["cropped_per_vehicle_fuel", "total_fuel_l_cropped", "BDAE"])

results = {}

for col in target_cols:
    results[col] = get_sa_results(
        results_df,
        problem,
        col,
        second_order=second_order,
    )

In [ ]:
# build the results dataframe for S1 and ST
sobol_df = pl.DataFrame(
    {
        "names": problem["names"],
        **{
            f"{col}_{sa_type}": results[col][sa_type]
            for col in target_cols
            for sa_type in ["S1", "ST", "S1_conf", "ST_conf"]
        },
    }
)

## Plots of Results

In [ ]:
sobol_df.sort("average_travel_time_ST", descending=True)[
    ["names", "average_travel_time_ST", "average_travel_time_ST_conf"]
]

In [ ]:
sobol_df.sort("average_delay_ST", descending=True)[
    ["names", "average_delay_ST", "average_delay_ST_conf"]
]

In [ ]:
sobol_df.sort("delay_ratio_ST", descending=True)[
    ["names", "delay_ratio_ST", "delay_ratio_ST_conf"]
]

In [ ]:
sobol_df.sort("cropped_per_vehicle_fuel_ST", descending=True)[
    [
        "names",
        "cropped_per_vehicle_fuel_ST",
        "cropped_per_vehicle_fuel_ST_conf",
        "cropped_per_vehicle_fuel_S1",
        "cropped_per_vehicle_fuel_S1_conf",
    ]
]

In [ ]:
from src.plotting.bar import plot_bars
from src.plotting.latex import ORDERED_VARIABLES

sobol_df = sobol_df.with_columns(
    pl.col("names").map_dict(ORDERED_VARIABLES).alias("order")
)

In [ ]:
fig = plot_bars(
    sobol_df,
    plot_columns=[
        "cropped_per_vehicle_fuel",
        "average_travel_time",
        # "average_fc",
        "average_delay",
        "delay_ratio",
    ],
    pretty_columns=[
        "Per-Vehicle Fuel Consumption",
        "Average Travel Time",
        "Average Delay",
        "Delay Ratio",
    ],
    sort_columns=["order"],
    variable_column="names",
    use_latex=True,
)

# display the figure as an image
from IPython.display import Image

Image(fig.to_image(format="png", width=800, height=1200))

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,
    x="FleetComposition",
    y="total_fuel_l_cropped",
    # color="calibration_passed",
    # color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
# approximating the relationship between fuel consumption and fleet composition with a linear model

from scipy.stats import pearsonr


print(
    pearsonr(
        results_df["FleetComposition"].to_numpy(),
        results_df["total_fuel_l_cropped"].to_numpy(),
    )
)

In [ ]:
results_df[["total_fuel_l_cropped", "cropped_per_vehicle_fuel"]].quantile(0.95)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="impatience",
    y="total_fuel_l_cropped",
    # color="calibration_passed",
    # color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,
    x="FleetComposition",
    y="delay_ratio",
    # color="calibration_passed",
    # color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
print(
    pearsonr(
        results_df["FleetComposition"].to_numpy(), results_df["delay_ratio"].to_numpy()
    )
)

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="BDAE",
    y="average_speed",
)


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="impatience",
    y="average_delay",
    color="cropped_per_vehicle_fuel",
    color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="Accel",
    y="cropped_per_vehicle_fuel",
    color="average_delay",
    color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="Tau",
    y="cropped_per_vehicle_fuel",
    color="average_delay",
    color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
import plotly.express as px
import plotly.graph_objects as go


# scatter plot decel vs. total fuel consumption
fig = px.scatter(
    results_df,  # .filter((pl.col("speedFactor") - 1.15).abs() < 0.05),
    x="Accel",
    y="BDAE",
    # color="average_delay",
    color_continuous_scale=px.colors.diverging.Tealrose,
)


fig.show()

In [ ]:
px.density_heatmap(
    results_df.to_pandas(),
    x="speedDev",
    y="speedFactor",
    z="cropped_per_vehicle_fuel",
    histfunc="avg",
)

In [ ]:
results_df[
    [
        "average_speed",
        "speedFactor",
        "speedDev",
        "cropped_per_vehicle_fuel",
        "impatience",
        "BDAE",
    ]
].corr()

In [ ]:
import plotly.graph_objects as go

col = "average_travel_time_ST"

sobol_df = sobol_df.sort(
    col,
)

fig = go.Figure(
    data=[
        go.Bar(
            name=col,
            y=sobol_df["names"],
            x=sobol_df[col],
            orientation="h",
            marker=dict(
                color="rgba(0, 0, 0, 0.4)",
                line=dict(color="rgba(0, 0, 0, 1.0)", width=2),
                pattern_shape="x",
            ),
        )
    ]
)

fig.update_layout(
    yaxis=dict(
        # dtick=0.1, tickangle=45,
        showgrid=True,
        tickvals=sobol_df["names"],
        minor_showgrid=True,
    ),
)


fig.show()

In [ ]:
import plotly.graph_objects as go

col = "total_fuel_l_cropped_ST"

sobol_df = sobol_df.sort(
    col,
)

fig = go.Figure(
    data=[
        go.Bar(
            name=col,
            y=sobol_df["names"],
            x=sobol_df[col],
            orientation="h",
            marker=dict(
                color="rgba(0, 0, 0, 0.4)",
                line=dict(color="rgba(0, 0, 0, 1.0)", width=2),
                pattern_shape="x",
            ),
        )
    ]
)

fig.update_layout(
    yaxis=dict(
        # dtick=0.1, tickangle=45,
        showgrid=True,
        tickvals=sobol_df["names"],
        minor_showgrid=True,
    ),
)


fig.show()

## Parallel Coordinates Plot of Sensitivity Analysis

## Distribution of Outputs

In [ ]:
results_df["total_fuel_l_cropped"].skew(), results_df["total_fuel_l_cropped"].kurtosis()

In [ ]:
results_df["average_travel_time"].skew(), results_df["average_travel_time"].kurtosis()

In [ ]:
results_df["delay_ratio"].skew(), results_df["delay_ratio"].kurtosis()

In [ ]:
from src.plotting.bar import plot_ecdfs


def open_res_df(path):
    return (
        pl.read_parquet(path / "results.parquet")
        .with_columns(
            total_fuel_l_cropped=pl.col("total_energy")
            / SUMO_GASOLINE_GRAM_TO_JOULE
            / SUMO_GASOLINE_GRAM_PER_LITER
        )
        .with_columns(
            cropped_per_vehicle_fuel=pl.col("total_fuel_l_cropped")
            / pl.col("total_vehicles_emissions")
        )
    )


fig = plot_ecdfs(
    plot_columns=[
        "total_fuel_l_cropped",
        "average_travel_time",
        "average_delay",
        "delay_ratio",
    ],
    pretty_columns=[
        "Total Fuel Consumption [L]",
        "Average Travel Time [s]",
        "Average Delay [s]",
        "Ratio of Delay",
    ],
    ecdf_mapping={
        "SA 2 Samples": (results_df, "black"),
        "SA 1 Samples": (
            open_res_df(
                ROOT / "tmp/SobolSensitivityAnalysisCars-2048/11.26.2023_15.04.49"
            ),
            "rgba(229, 185, 173, 1)",
        ),
    },
)

fig.show()

In [ ]:
failure_df = results_df.filter(~pl.col("calibration_passed"))
pass_df = results_df.filter(pl.col("calibration_passed"))

In [ ]:
import scipy.stats as stats

stats.ks_2samp(failure_df["average_delay"], pass_df["average_delay"])

In [ ]:
stats.ks_2samp(failure_df["total_fuel_l_cropped"], pass_df["total_fuel_l_cropped"])

In [ ]:
stats.ks_2samp(failure_df["average_travel_time"], pass_df["average_travel_time"])

In [ ]:
# test to see if the failure_df is statistically different from the success_df
import scipy.stats as stats

stats.ks_2samp(failure_df["average_delay"], pass_df["average_delay"])

# plot the two
fig = plot_ecdfs(
    plot_columns=[
        "total_fuel_l_cropped",
        "average_travel_time",
        "average_delay",
        "delay_ratio",
    ],
    pretty_columns=[
        "Total Fuel Consumption [L]",
        "Average Travel Time [s]",
        "Average Delay [s]",
        "Ratio of Delay",
    ],
    ecdf_mapping={
        "Passed": (pass_df, "rgba(0, 147, 146, 1)"),
        "Failed": (failure_df, "rgba(229, 185, 173, 1)"),
    },
)

fig.show()

In [ ]:
1 - results_df['calibration_passed'].mean()

In [ ]:
1 - results_df.filter(
    pl.col('average_delay') > pl.col('average_delay').quantile(0.99)
)['calibration_passed'].mean()

In [ ]:
(failure_df['FleetComposition'] > results_df['FleetComposition'].mean()).mean()